# Import a Molecules/Atom from cartesian coordinates:
**Task:** 
1. Create a Molecule object from its cartesian coordinates
2. Ask for its atomic and molecular properties

## [Water molecule](https://en.wikipedia.org/wiki/Properties_of_water) ($H_{2}O$):

<img 
src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b7/H2O_2D_labelled.svg/2560px-H2O_2D_labelled.svg.png"
alt="water" 
width="200" 
height="100" 
/>

### Properties:
- Number of molecules: 1
- Number of atoms: 3
- Atomic symbols: O, H, H
- Atomic masses [g/mol]: 
    - O: 15.999
    - H: 1.008
    - H: 1.008
- Molar mass [g/mol]: 18.015
- Cartesian coordinates [Angstrom]:
    ```XYZ
    O    0.00000    0.00000    0.00000
    H    0.58708    0.75754    0.00000
    H    0.58708   -0.75754    0.00000
    ```
- Bond distance [Angstrom]:
    - H-O: 0.9584 (95.84 pm)
    - H-H: 1.5151
- Bond angle [Degrees]:
    - H-O-H: 104.45
    - H-H-O: 37.78

In [3]:
# crating the path (PYTHONPATH) to our module.
# assuming that our 'src' directory is out ('..') of our current directory 

import os
import sys
module_path = os.path.abspath(os.path.join('../../src'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
from src.base_molecule import Cluster, Molecule
# Creates a molecule. You can use a list, a dictionary (the key MUST be "atoms")
# or another molecule object (see below)

# 1 Option
water=[("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)]
water_molecule = Molecule(water)

# 2 Option
water_dict = {"atoms": [("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)]}
water_molecule = Molecule(water_dict)



In [5]:
## See initial molecule.

import py3Dmol

water_molecule = Molecule(water_dict)
water_molecule = water_molecule.translate(0, x=1.5, y=1.5, z=1.5)
bi_water = water_molecule.add_fragments(water_dict)


xyz_view = py3Dmol.view(width=300,height=200)
xyz_view.addModel(str(bi_water),'xyz')
xyz_view.setStyle({'stick':{}})

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [8]:
import sys

sys.path.append("../")

from src.search_configuration import SearchConfig


#################################################
# Configuración del sistema de trabajo
#################################################

h2o = Cluster(water, water)

################################################
# Configuración de la busqueda
###############################################


search_config = SearchConfig(h2o)
search_config.shgo() #da, shgo

/home/danian/famaf/ascec/src/search_configuration.py:49: UserWarning: Center of Mass of the fragments 0 and 1 are overlapping. Then 1 is move 0.48168533595111107
  warnings.warn(message)


converged SCF energy = -149.860272959031
converged SCF energy = -149.860272959031
converged SCF energy = -149.860231037151
converged SCF energy = -149.860665338045
converged SCF energy = -149.860620897562
converged SCF energy = -149.860613113798
converged SCF energy = -149.85986431817


KeyboardInterrupt: 

In [27]:
#Visualization
def visualization(serie_string : str, n_atoms):
    #Approximately 3 bohr radius for each atom
    sphere_radius = n_atoms*1.5*0.5
    xyz_view = py3Dmol.view(width=1000,height=700)
    xyz_view.addModelsAsFrames(serie_string,'xyz')
    xyz_view.setStyle({'sphere': {'radius': 0.8}})
    xyz_view.addSphere({'center': {'x':0, 'y':0, 'z':0}, 
        'radius': sphere_radius , 
        'color' :'yellow',
        'alpha': 0.5,
        })

    xyz_view.animate({'loop': "forward", 'speed': 1, 'reps': 3})
    xyz_view.zoomTo()
    xyz_view.show()

#El archivo configurations.xyz es guardado en la dirección de este jupyter.notebook
with open("configurations.xyz","r") as outxyz:
    serie_string = outxyz.read()
    visualization(serie_string,h2o._total_atoms)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
# Atomic Simulation Environment
# https://wiki.fysik.dtu.dk/ase/index.html
# !pip install --upgrade --user ase


# ChemML
# https://hachmannlab.github.io/chemml/index.html
# !pip install chemml

In [ ]:
# NGLview
# https://github.com/nglviewer/nglview
# !pip install nglview

# ---------------------------------------
# pytraj 
# https://amber-md.github.io/pytraj/latest/index.html
# !pip install pytraj
